# Wide Binaries Selection Script
---

This script's purpose is to select white dwarves in wide binaries using information from Gaia and SDSS. It's a work in process.

## 01 - SDSS Metadata

In [1]:
# Install a pip package in the current Jupyter kernel
import sys

### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from tqdm import tqdm
import pandas as pd

### Query
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia

First, read in the data table from [Kepler et. al, 2019](https://arxiv.org/pdf/1904.01626.pdf) and extract the plate, mjd, and fiberID that will be used to identify the spectra.

In [2]:
df = pd.read_csv("data/sdssdr14-wd-catalog/Table2.csv")

ps = []
ms = []
fs = []

for i in range(len(df['P-M-F'])):
    tpmf = df['P-M-F'][i].split('-')
        
    ps.append(int(tpmf[0]))
    ms.append(int(tpmf[1]))
    fs.append(int(tpmf[2]))
        
df['plate'] = ps
df['mjd'] = ms
df['fiber'] = fs

C:\Users\arsen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Now, get the spectra from the above information.

In [3]:
def get_spectra(df):
    xids = []
    
    for i in tqdm (range(len(df['SNg']))):
        try:
            xid = SDSS.get_spectra_async(plate=df['plate'][i], fiberID=df['fiber'][i], mjd=df['mjd'][i])
            xids.append(xid)
        except:
            pass
    return xids
    
xids = get_spectra(df)

'''
from numpy import asarray
from numpy import save

if dothing:
    xids2 = np.array(xids)
    xids2.tofile('data/wd-spectra.txt', sep = ',')
    
print(len(xids))
'''

 57%|███████████████████████████████████████████▌                                | 21433/37406 [04:25<03:17, 80.69it/s]C:\Users\arsen\anaconda3\lib\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|████████████████████████████████████████████████████████████████████████████| 37406/37406 [06:24<00:00, 97.23it/s]


"\nfrom numpy import asarray\nfrom numpy import save\n\nif dothing:\n    xids2 = np.array(xids)\n    xids2.tofile('data/wd-spectra.txt', sep = ',')\n    \nprint(len(xids))\n"

We query the other information we'll need to cross-match with Gaia. Namely bestObjID. This step can be optimized, but I'm not sure how else to do it for now. Because there are too many datapoints (~900000) returned by the query I want to do, I'm splitting it into two and then stacking the tables.

In [4]:
query1 = "select s.bestObjID, s.specObjID, s.plate, s.mjd, s.fiberID \
    from SpecObj as s \
    where s.plate > 265\
    and s.plate < 3400\
    and s.class = \'STAR\'"
query2 = "select s.bestObjID, s.specObjID, s.plate, s.mjd, s.fiberID \
    from SpecObj as s \
    where s.plate > 3400\
    and s.plate < 9004\
    and s.class = \'STAR\'"
            
white_dwarf1= SDSS.query_sql(query1)
white_dwarf2= SDSS.query_sql(query2)

In [5]:
from astropy.table import Table, vstack

twhite_dwarf = vstack([white_dwarf1, white_dwarf2])
white_dwarf = twhite_dwarf

Now we can join the `(plate, mjd, fiberID)` tuple to the associated bestObjID. We do this by manually searching the returned dataset. This is basically a `join` command in SQL but I'm not sure how to do it in SQL.

In [6]:
id_tuple = np.array([df['plate'], df['mjd'], df['fiber']]).T
white_dwarf = twhite_dwarf.to_pandas()

i = np.where((id_tuple[:,2] == 360) & (id_tuple[:,0] == 54414))

In [7]:
def search(plate, fiber, mjd):
    urls = []
    drops = []
    
    for i in tqdm(range(len(plate))):
        notfound = False
        j = np.where((id_tuple[:,0] == plate[i]) & (id_tuple[:,1] == mjd[i]) & (id_tuple[:,2] == fiber[i]))
        
        try:
            k = j[0][0]
        except:
            notfound = True
            
        if not notfound: 
            try:
                urls.append(str(xids[k][0]).split(' ')[4])
            except:
                notfound = True
        if notfound:
            drops.append(i)
            
    return drops, urls

drops, urls = search(white_dwarf['plate'], white_dwarf['fiberID'], white_dwarf['mjd'])

100%|████████████████████████████████████████████████████████████████████████| 887342/887342 [03:13<00:00, 4579.93it/s]


In [8]:
print(len(drops))
print(len(white_dwarf) - len(df))

white_dwarf = white_dwarf.drop(drops)

print(len(white_dwarf))

white_dwarf['url'] = urls
white_dwarf.reset_index(inplace=True, drop=True)
print(white_dwarf[:10])

855752
849936
31590
             bestObjID     specObjID  plate    mjd  fiberID  \
0  1237664087534732151  3.038948e+18   2699  54414      525   
1  1237664087534797369  3.038930e+18   2699  54414      459   
2  1237664087534928362  3.038958e+18   2699  54414      561   
3  1237664089146064917  1.415271e+18   1257  52944       53   
4  1237664089146064948  1.415269e+18   1257  52944       47   
5  1237664089146065069  1.415257e+18   1257  52944        1   
6  1237664089682149405  1.415322e+18   1257  52944      240   
7  1237664089682149414  1.415318e+18   1257  52944      223   
8  1237664089682149435  1.415319e+18   1257  52944      228   
9  1237664089682346088  1.415308e+18   1257  52944      190   

                                                 url  
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
1  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
2  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
3  https://data.sdss.org/sas/dr14/sdss/spectro/re...  
4  https://

Finally, we save the data. `'data/white_dwarves.csv'` contains all the info needed to work with most of the white dwarves from Kepler. The actual number in this file is less than what's listed in Kepler. I think that's because some of the dwarves in question might not have spectra associated with it, but I'm not totally sure.

In [9]:
white_dwarf.to_csv('data/white_dwarves.csv')

## 02 - SDSS Metadata to Gaia Metadata

Now we can start working with the Gaia data. To start, we read in the wide dwarves catalog from [El-Badry et. al, 2021](https://ui.adsabs.harvard.edu/abs/2021MNRAS.506.2269E/abstract). 

In [10]:
hdul = fits.open('../external-dat/binaries/all_columns_catalog.fits')

In [11]:
hdul[1].data['source_id1']
hdul[1].data['source_id2']

array([4282339095720590720, 4089437344115586432, 4282722790916299648, ...,
       6056106923357411840, 6056106992113048960, 6056109088057213312],
      dtype=int64)

In [12]:
white_dwarf = pd.read_csv('data/white_dwarves.csv')
bestobjid = tuple(white_dwarf['bestObjID'])
print(bestobjid[1])

1237664087534797369


Now we query Gaia to get the Gaia `source_id`'s associated with the `bestObjID`'s that we got from above and tack this onto the white dwarves dataframe in a new column.

In [13]:
ADQL_CODE = "SELECT \
    sdss.original_ext_source_id as bestobjid,\
    gaia_source.source_id\
    FROM gaiaedr3.gaia_source \
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss\
    ON gaia_source.source_id = sdss.source_id      \
    WHERE sdss.original_ext_source_id IN {}\
".format(bestobjid)

In [14]:
job = Gaia.launch_job_async(ADQL_CODE,dump_to_file=False)
conversion = job.get_results()

print(conversion[:10])

INFO: Query finished. [astroquery.utils.tap.core]
     bestobjid           source_id     
------------------- -------------------
1237645943973609500     313189015283712
1237645943973675103     298895364329216
1237646586638631914 3219069845094872960
1237646587712570275 3219421310859198080
1237646792773403258 3113487561710626560
1237646793309750322 3119769483955377408
1237646793310209040 3125530924886250752
1237646793310405381 3125541022348840448
1237646793310537053 3113532126290835072
1237646794374906022 3234158477521752704


In [15]:
white_dwarf_test = white_dwarf
print(len(white_dwarf))
print(len(conversion))

31590
31273


In [16]:
source_ids = []
drops = []

for i in range(len(white_dwarf_test)):
    notfound = False
    j = np.where(white_dwarf_test['bestObjID'][i] == conversion['bestobjid'])
    
    try:
        k = j[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            source_ids.append( conversion['source_id'][k] )
        except:
            notfound = True
    if notfound:
        drops.append(i)

In [17]:
white_dwarf = white_dwarf.drop(drops)
white_dwarf['source_id'] = source_ids
white_dwarf.reset_index(inplace=True, drop=True)

print(white_dwarf[:10])

   Unnamed: 0            bestObjID     specObjID  plate    mjd  fiberID  \
0           0  1237664087534732151  3.038948e+18   2699  54414      525   
1           1  1237664087534797369  3.038930e+18   2699  54414      459   
2           2  1237664087534928362  3.038958e+18   2699  54414      561   
3           3  1237664089146064917  1.415271e+18   1257  52944       53   
4           4  1237664089146064948  1.415269e+18   1257  52944       47   
5           5  1237664089146065069  1.415257e+18   1257  52944        1   
6           6  1237664089682149405  1.415322e+18   1257  52944      240   
7           7  1237664089682149414  1.415318e+18   1257  52944      223   
8           8  1237664089682149435  1.415319e+18   1257  52944      228   
9           9  1237664089682346088  1.415308e+18   1257  52944      190   

                                                 url            source_id  
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...  3413138973156922368  
1  https://data.sdss.o

## 03 - Wide Binary Search

Finally, we search along `source_id` and log which recorded object in the dataset is the white dwarf. *TODO: add more info on the main sequence object*

In [18]:
wide_binary = []
not_wide_binary = []

for i in tqdm(range(len(white_dwarf))):
    if white_dwarf['source_id'][i] in hdul[1].data['source_id1']:
        wide_binary.append( (i, 1) )
    elif white_dwarf['source_id'][i] in hdul[1].data['source_id2']:
        wide_binary.append( (i, 2) )
    else:
        not_wide_binary.append( i )

100%|████████████████████████████████████████████████████████████████████████████| 31230/31230 [50:27<00:00, 10.32it/s]


In [21]:
wide_binaries = white_dwarf.drop(not_wide_binary)
wide_binaries['object_num'] = np.array(wide_binary).T[1]
wide_binaries.reset_index(inplace=True, drop=True)

twide_binaries1 = wide_binaries

print(wide_binaries[:10])

   Unnamed: 0            bestObjID     specObjID  plate    mjd  fiberID  \
0          75  1237664091830812971  3.280978e+18   2914  54533      383   
1         155  1237672005834114035  3.176330e+18   2821  54393      605   
2         159  1237658608219062381  1.075246e+18    955  52409       41   
3         207  1237662336250544181  1.575136e+18   1399  53172        6   
4         294  1237661125071732829  1.782346e+18   1583  52941      167   
5         299  1237661125074944335  1.784700e+18   1585  52962      542   
6         356  1237661871325053068  1.621408e+18   1440  53084      408   
7         395  1237662193452712021  1.632565e+18   1450  53120       38   
8         451  1237661871347204180  1.562909e+18   1388  53119      583   
9         511  1237665126927237227  2.372332e+18   2107  53786      221   

                                                 url            source_id  \
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...   697212800055168896   
1  https://data.sdss

In [23]:
wide_binaries = twide_binaries1
ms_source = []

for i in tqdm (range(len(wide_binaries))):
    if wide_binaries['object_num'][i] == 1:
        j = np.where(hdul[1].data['source_id1'] == wide_binaries['source_id'][i])[0][0]
        ms_source.append(hdul[1].data['source_id2'][j])
    else:
        j = np.where(hdul[1].data['source_id2'] == wide_binaries['source_id'][i])[0][0]
        ms_source.append(hdul[1].data['source_id1'][j])
        
#wide_binaries = twide_binaries2

100%|████████████████████████████████████████████████████████████████████████████████| 851/851 [00:34<00:00, 24.96it/s]


In [26]:
wide_binaries['mainseq_sourc0e'] = ms_source

print(wide_binaries[:10])

   Unnamed: 0            bestObjID     specObjID  plate    mjd  fiberID  \
0          75  1237664091830812971  3.280978e+18   2914  54533      383   
1         155  1237672005834114035  3.176330e+18   2821  54393      605   
2         159  1237658608219062381  1.075246e+18    955  52409       41   
3         207  1237662336250544181  1.575136e+18   1399  53172        6   
4         294  1237661125071732829  1.782346e+18   1583  52941      167   
5         299  1237661125074944335  1.784700e+18   1585  52962      542   
6         356  1237661871325053068  1.621408e+18   1440  53084      408   
7         395  1237662193452712021  1.632565e+18   1450  53120       38   
8         451  1237661871347204180  1.562909e+18   1388  53119      583   
9         511  1237665126927237227  2.372332e+18   2107  53786      221   

                                                 url            source_id  \
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...   697212800055168896   
1  https://data.sdss

In [27]:
ADQL_CODE = "SELECT \
    sdss.original_ext_source_id as bestobjid,\
    gaia_source.source_id\
    FROM gaiaedr3.gaia_source \
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss\
    ON gaia_source.source_id = sdss.source_id      \
    WHERE gaia_source.source_id IN {}\
".format(tuple(ms_source))

In [28]:
job = Gaia.launch_job_async(ADQL_CODE,dump_to_file=False)
d = job.get_results()

print(len(wide_binaries))
print(d)

INFO: Query finished. [astroquery.utils.tap.core]
851
     bestobjid           source_id     
------------------- -------------------
1237670016735838345    7030178564427008
1237653652920991870   78023239188201216
1237679460828905657   95187985791901440
1237666276352983081  104000541424155520
1237670458578370766  127904645766742912
1237670960554246879  136780586525373184
1237670962167152981  143563473757148672
1237660761073779312  148308244388792576
1237661120240615745  235840678609579904
1237680070713409692  290411536638937472
                ...                 ...
1237661388159582340 4574942916808430336
1237649961381658756 5168317765719259648
1237652901307220080 5169712118261363968
1237649964066078921 5169997858141425920
1237679438279802966 5177858850883734144
1237652934033801258 6898489884295412352
1237652935644611057 6898870693275487232
1237666183476543554 6899127089938464512
1237652933493785651 6907030890620245632
1237652935103283225 6908406487039810432
1237652935107019325 691080

In [29]:
ms_bestobjid = []
drops = []

for i in tqdm (range(len(wide_binaries))):
    notfound = False
    j = np.where(d['source_id'] == wide_binaries['mainseq_source'][i])
    
    try:
        k = j[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            ms_bestobjid.append(d['bestobjid'][j][0])
        except:
            notfound = True
    if notfound:
        drops.append(i)    

100%|██████████████████████████████████████████████████████████████████████████████| 851/851 [00:00<00:00, 4865.91it/s]


In [30]:
wide_binaries = wide_binaries.drop(drops)
wide_binaries['mainseq_bestobjid'] = ms_bestobjid
wide_binaries.reset_index(inplace=True, drop=True)
twide_binaries2 = wide_binaries

print(wide_binaries[:10])

   Unnamed: 0            bestObjID     specObjID  plate    mjd  fiberID  \
0          75  1237664091830812971  3.280978e+18   2914  54533      383   
1         155  1237672005834114035  3.176330e+18   2821  54393      605   
2         159  1237658608219062381  1.075246e+18    955  52409       41   
3         207  1237662336250544181  1.575136e+18   1399  53172        6   
4         294  1237661125071732829  1.782346e+18   1583  52941      167   
5         299  1237661125074944335  1.784700e+18   1585  52962      542   
6         356  1237661871325053068  1.621408e+18   1440  53084      408   
7         395  1237662193452712021  1.632565e+18   1450  53120       38   
8         451  1237661871347204180  1.562909e+18   1388  53119      583   
9         511  1237665126927237227  2.372332e+18   2107  53786      221   

                                                 url            source_id  \
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...   697212800055168896   
1  https://data.sdss

In [32]:
print(min(wide_binaries['mainseq_bestobjid']))
print(max(wide_binaries['mainseq_bestobjid']))

1237648702985666869
1237680529738105575


In [33]:
SDSS_CODE1 = """select bestObjID, specObjID, plate, fiberID, mjd 
    from dbo.SpecObjAll
    where bestObjID > 1237648702985666868
    and bestObjID < 1237660529738105365
    and class = 'STAR'"""
SDSS_CODE2 = """select bestObjID, specObjID, plate, fiberID, mjd 
    from dbo.SpecObjAll
    where bestObjID > 1237660529738105365
    and bestObjID < 1237670529738105366
    and class = 'STAR'"""
SDSS_CODE3 = """select bestObjID, specObjID, plate, fiberID, mjd 
    from dbo.SpecObjAll
    where bestObjID > 1237670529738105366
    and bestObjID < 1237680529738105576
    and class = 'STAR'"""

In [34]:
import sdss

convert1= SDSS.query_sql(SDSS_CODE1)
convert2= SDSS.query_sql(SDSS_CODE2)
convert3= SDSS.query_sql(SDSS_CODE3)

In [35]:
print(len(convert1))
print(len(convert2))
print(len(convert3))

314976
480510
197704


In [36]:
twide_convert = vstack([convert1, convert2, convert3])
wide_convert = twide_convert
print(wide_convert)

     bestObjID            specObjID        plate fiberID  mjd 
------------------- ---------------------- ----- ------- -----
1237648702985732625  4.522953870205231e+18  4017     778 55329
1237648702985798284  4.522960467274998e+18  4017     802 55329
1237648702985863176  3.501933818691195e+17   311     140 51665
1237648702985928999 3.5018183699702784e+17   311      98 51665
1237648702986059806 1.0393068094179512e+18   923     368 52404
1237648702986060928  4.519463475592798e+18  4014     368 55630
1237648702986125333 4.5194527553544274e+18  4014     329 55630
1237648702986125397 1.0393183542900429e+18   923     410 52404
1237648702986125619  4.523004447740109e+18  4017     962 55329
1237648702986256396  3.513629877658194e+17   312     299 51689
                ...                    ...   ...     ...   ...
1237680529737909863  7.089900394482999e+18  6297     395 56218
1237680529737909895  7.089889674244628e+18  6297     356 56218
1237680529737910406  8.617811457910743e+18  7654     63

In [50]:
#
#  Reset from here
#

wide_binaries = twide_binaries2
print(wide_binaries[:10])

   Unnamed: 0            bestObjID     specObjID  plate    mjd  fiberID  \
0          75  1237664091830812971  3.280978e+18   2914  54533      383   
1         155  1237672005834114035  3.176330e+18   2821  54393      605   
2         159  1237658608219062381  1.075246e+18    955  52409       41   
3         207  1237662336250544181  1.575136e+18   1399  53172        6   
4         294  1237661125071732829  1.782346e+18   1583  52941      167   
5         299  1237661125074944335  1.784700e+18   1585  52962      542   
6         356  1237661871325053068  1.621408e+18   1440  53084      408   
7         395  1237662193452712021  1.632565e+18   1450  53120       38   
8         451  1237661871347204180  1.562909e+18   1388  53119      583   
9         511  1237665126927237227  2.372332e+18   2107  53786      221   

                                                 url            source_id  \
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...   697212800055168896   
1  https://data.sdss

In [51]:
def search(bestObjID):
    sourceobjid = []
    plate = []
    mjd = []
    fiberid = []
    
    url = []
    
    drops = []
    
    for i in tqdm(range(len(bestObjID))):
        notfound = False
        j = np.where(( bestObjID[i] == wide_convert['bestObjID'] ))
        
        try:
            k = j[0][0]
        except:
            notfound = True
            
        if not notfound: 
            try:
                sourceobjid.append(wide_convert['specObjID'][k])
                plate.append(wide_convert['plate'][k])
                mjd.append(wide_convert['mjd'][k])
                fiberid.append(wide_convert['fiberID'][k])
            except:
                notfound = True
        if notfound:
            drops.append(i)  
            
    for i in tqdm(range(len(sourceobjid))):
        xid = SDSS.get_spectra_async(plate=plate[i], fiberID=fiberid[i], mjd=mjd[i])
        url.append(str(xid[0]).split(' ')[4])
    
    
    return drops, sourceobjid, plate, mjd, fiberid, url

drops, sourceobjid, plate, mjd, fiberID, url = search( wide_binaries['mainseq_bestobjid'] )

  0%|                                                                                           | 0/49 [00:00<?, ?it/s]C:\Users\arsen\anaconda3\lib\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|█████████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 257.88it/s]


In [52]:
print(len(url))

wide_binaries = wide_binaries.drop(drops)

print(len(wide_binaries))

wide_binaries['mainseq_sourceobjid'] = sourceobjid
wide_binaries['mainseq_plate'] = plate
wide_binaries['mainseq_mjd'] = mjd
wide_binaries['mainseq_fiberID'] = fiberID
wide_binaries['mainseq_url'] = url

wide_binaries.reset_index(inplace=True, drop=True)
print(wide_binaries[:10])

49
49
   Unnamed: 0            bestObjID     specObjID  plate    mjd  fiberID  \
0         511  1237665126927237227  2.372332e+18   2107  53786      221   
1         512  1237665126927237234  2.372323e+18   2107  53786      189   
2        4341  1237658800959520914  1.139488e+18   1012  52649      282   
3        4860  1237663783656882228  4.312502e+17    383  51818      111   
4        5210  1237661949719937087  1.819464e+18   1616  53169       36   
5        7811  1237668331488739458  3.091785e+18   2746  54232      231   
6        8297  1237657611260133456  3.575915e+18   3176  54832      206   
7        8751  1237658918533070860  1.169982e+18   1039  52707      624   
8        8752  1237658918533070872  1.171035e+18   1040  52722      361   
9        9402  1237664836462575849  2.169737e+18   1927  53321      464   

                                                 url            source_id  \
0  https://data.sdss.org/sas/dr14/sdss/spectro/re...   757911884925087104   
1  https://dat

In [53]:
#wide_binaries = pd.read_csv('data/wide_binaries.csv')

In [54]:
xids = []
drop = []

for i in tqdm (range(len(wide_binaries))):
    try:
        xid = SDSS.get_spectra_async(plate=wide_binaries['mainseq_plate'][i], fiberID=wide_binaries['mainseq_fiberID'][i], mjd=wide_binaries['mainseq_mjd'][i])
        xids.append(str(xid).split(' ')[4])
    except:
        drop.append(i)
    
print(len(wide_binaries))
print(len(xids))

  0%|                                                                                           | 0/49 [00:00<?, ?it/s]C:\Users\arsen\anaconda3\lib\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|█████████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 533.76it/s]

49
49


In [55]:
wide_binaries['mainseq_url'] = xids

In [56]:
drops = []

for i in tqdm(range(len(wide_binaries))):
    found = True
    j = np.where(wide_binaries['specObjID'][i] == wide_binaries['mainseq_sourceobjid'])
    
    try:
        k = j[0][0]
    except:
        notfound = False
        
    if found:
        drops.append(k)
    

100%|████████████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 2764.68it/s]


In [57]:
wide_binaries = wide_binaries.drop(drops)
wide_binaries.reset_index(inplace=True, drop=True)


In [58]:
print(wide_binaries)

    Unnamed: 0            bestObjID     specObjID  plate    mjd  fiberID  \
0          511  1237665126927237227  2.372332e+18   2107  53786      221   
1         4341  1237658800959520914  1.139488e+18   1012  52649      282   
2         4860  1237663783656882228  4.312502e+17    383  51818      111   
3         5210  1237661949719937087  1.819464e+18   1616  53169       36   
4         8297  1237657611260133456  3.575915e+18   3176  54832      206   
5         9402  1237664836462575849  2.169737e+18   1927  53321      464   
6        10114  1237657191447330936  7.781569e+17    691  52199      582   
7        11717  1237661967967125574  1.632721e+18   1450  53120      606   
8        12793  1237661966353498127  1.626962e+18   1445  53062      133   
9        13743  1237657609107341402  8.523090e+17    757  52238       10   
10       15220  1237661852015198226  1.516734e+18   1347  52823      535   
11       16522  1237665567684755609  2.420764e+18   2150  54510      288   
12       173

In [59]:
wide_binaries.to_csv('data/wide_binaries.csv')